# Dataset Creation Notebook
#### Step 0. Notebook Setup

Purpose: Import required libraries and suppress unnecessary warnings so outputs stay readable.

In [2]:
## Dataset Module
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#### Step 1. Load Raw Datasets

Purpose: Load the raw insurance policy data and claims reference data from CSV files.
These datasets are the starting point for all downstream feature engineering.

In [3]:
## load datasets
insurance = pd.read_csv('../../data/input/exp/Motor_vehicle_insurance_data.csv', delimiter=";")
claims =  pd.read_csv('../../data/input/exp/sample_type_claim.csv', delimiter=';')

#### Step 2. Remove Columns Unavailable for New Customers

Purpose: Drop variables that depend on future information or historical outcomes.
These fields would not exist at prediction time for a new customer and would cause data leakage.

In [4]:
## columns to drop which won't be available for new customers
cols_to_drop = ['ID','Date_last_renewal','Date_start_contract','Date_next_renewal', 
                'Seniority', 'Policies_in_force', 'Max_policies', 
                'Max_products','Lapse', 'Date_lapse', 'Cost_claims_year', 
                'N_claims_year', 'N_claims_history', 'R_Claims_history']

insurance_new = insurance.drop(columns=cols_to_drop)

#### Step 3. Define Categorical Variable Mappings

Purpose: Convert encoded categorical variables into human-readable labels.
This improves interpretability and avoids passing opaque numeric codes downstream.

Variable encodings:

- 1. Distribution Channel - 0: Agent, 1: Broker
- 2. Payment - 0: half-yearly, 1: annually
- 3. Type_risk - 1: motorbikes, 2:vans, 3:passenger cars, 4:agricultural vehicles
- 4. Area - 0:rural, 1:urban
- 5. Second_driver - 0: no, 1: yes
- 6. Type_fuel - P: petrol, D: diesel 

In [ ]:
### Categorical Variables Mapping Functions

# Function to map Distribution Channel
def distribution_channel(x):
    if x == 0:
        return 'Agent'
    elif x == 1:
        return 'Broker'
    else:
        raise ValueError(f'Invalid Distribution_channel value: {x}. Expected 0 or 1.')

# Function to map Payment
def payment_method(x):
    if x == 0:
        return 'half-yearly'
    elif x == 1:
        return 'annually'
    else:
        raise ValueError(f'Invalid Payment value: {x}. Expected 0 or 1.')
    
# Function to map Type_risk
def type_risk(x):
    if x == 1:
        return 'motorbikes'
    elif x == 2:
        return 'vans'
    elif x == 3:
        return 'passenger cars'
    elif x == 4:
        return 'agricultural vehicles'
    else:
        raise ValueError(f'Invalid Type_risk value: {x}. Expected 1, 2, 3, or 4.')
    
# Function to map Area
def area_type(x):
    if x == 0:
        return 'rural'
    elif x == 1:
        return 'urban'
    else:
        raise ValueError(f'Invalid Area value: {x}. Expected 0 or 1.')
    
# Function to map Second_driver
def second_driver(x):
    if x == 0:
        return 'No'
    elif x == 1:
        return 'Yes'
    else:
        raise ValueError(f'Invalid Second_driver value: {x}. Expected 0 or 1.')
    
# Function to map Type_fuel
def type_fuel(x):
    if x == 'P':
        return 'petrol'
    elif x == 'D':
        return 'diesel'
    else:
        raise ValueError(f'Invalid Type_fuel value: {x}. Expected "P" or "D".')
    


#### Step 4. Apply Categorical Transformations

Purpose: Replace encoded categorical values in the dataset using the mapping functions defined above.

In [6]:
# Apply the functions to the insurance dataframe
insurance_new['Distribution_channel'] = insurance_new['Distribution_channel'].apply(distribution_channel)
insurance_new['Payment'] = insurance_new['Payment'].apply(payment_method)
insurance_new['Type_risk'] = insurance_new['Type_risk'].apply(type_risk)    
insurance_new['Area'] = insurance_new['Area'].apply(area_type)
insurance_new['Second_driver'] = insurance_new['Second_driver'].apply(second_driver)
insurance_new['Type_fuel'] = insurance_new['Type_fuel'].apply(type_fuel)

#### Step 5. Convert Date Columns to Datetime

Purpose: Ensure all date variables are stored in a proper datetime format.
Only columns starting with Date_ are converted, preventing accidental type changes.

In [ ]:

## convert date columns to datetime format for variables that start with 'Date_'
date_columns = [col for col in insurance_new.columns if col.startswith('Date_')]
for col in date_columns:
    insurance_new[col] = pd.to_datetime(insurance_new[col],format='%d/%m/%Y', errors='coerce')
    


#### Step 6. Sanity Check Final Dataset

Purpose: Confirm that the dataset loads correctly and inspect the first few rows.

In [ ]:
# display the first few rows of the final dataset
insurance_new.head()


#### Step 7. Module Execution Check (Optional)

Purpose: Allow this notebook or script to be imported safely without executing inspection logic.

In [ ]:
# Display the first few rows of the final dataset
if __name__ == "__main__":
    print("Dataset module loaded successfully.")
